In [1]:
# %load_ext autorelaod
# %autoreload 2

import sys
sys.path.append('./../RWKV-v4neo/src')
import torch 
from torch.utils import data
from enwik8_dataset import EnWik8
import json
import argparse
import os
from torch import nn
from lightning import seed_everything
seed_everything(42)

/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42

In [2]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)
def set_environ(env_vars: dict):
    for k, v in env_vars.items():
        os.environ[k] = v

In [3]:
data_path = '/system/user/beck/pwbeck/projects/rwkv/RWKV-LM/data/enwik8'
args = load_json('./debug_args.json')
args = argparse.Namespace(**args)
env_vars = load_json('./debug_environ.json')
set_environ(env_vars)

In [4]:
batch_size = 12

In [5]:
dataset = EnWik8(datafile=data_path)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=True)
args.vocab_size = dataset.vocab_size

Data has 99621832 tokens, 6064 vocab size.


In [6]:
batch = next(iter(dataloader))
x, y = batch
x.shape, y.shape

(torch.Size([12, 512]), torch.Size([12, 512]))

In [7]:
x

tensor([[16,  1,  1,  ...,  2, 84, 71],
        [ 2,  2,  2,  ..., 23, 25, 23],
        [20, 30, 17,  ..., 90, 67, 80],
        ...,
        [75, 80, 72,  ..., 67, 86,  2],
        [74, 86, 86,  ..., 53, 82, 67],
        [74, 67, 84,  ..., 86, 74, 71]])

### RWKV-v4neo model

In [8]:
from model import RWKV, Block, RWKV_ChannelMix, RWKV_TimeMix
model = RWKV(args)

RWKV_MY_TESTING 


Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv_512/build.ninja...
Building extension module wkv_512...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv_512...


In [9]:
output = model(x)

In [10]:
output.shape

torch.Size([12, 512, 6064])

In [11]:
output

tensor([[[-0.2103, -0.0397,  1.0679,  ...,  0.4574,  0.4023,  0.7730],
         [-0.0186, -0.4040,  0.1959,  ...,  0.3247, -0.1660,  0.1206],
         [ 0.1313, -0.1927,  0.0643,  ...,  0.2014, -0.0028, -0.0101],
         ...,
         [-0.1995,  0.3153, -0.5761,  ...,  0.2511,  0.2075, -0.6945],
         [ 0.5926,  0.1980, -0.1112,  ..., -0.5263,  0.4925, -0.1763],
         [-0.3525,  0.1504,  0.0842,  ..., -0.2548,  0.8232, -0.7318]],

        [[-0.2065,  0.3040, -0.5562,  ...,  0.3792,  0.2712, -0.6763],
         [-0.2807,  0.4576, -0.5695,  ...,  0.4815,  0.4311, -0.6347],
         [-0.2748,  0.4582, -0.5705,  ...,  0.4939,  0.4313, -0.6318],
         ...,
         [-0.1225, -0.1315,  0.3247,  ..., -0.2733, -0.6395,  0.6690],
         [ 0.1875,  0.1542, -0.5463,  ...,  0.2903,  0.0968, -0.4922],
         [-0.1231, -0.1379,  0.3264,  ..., -0.2627, -0.6288,  0.6606]],

        [[-0.4080,  0.6201,  1.3100,  ..., -0.0469,  0.3698, -0.5439],
         [ 0.2462, -0.1861, -0.1573,  ..., -0

In [12]:
output[0, 0, :]

tensor([-0.2103, -0.0397,  1.0679,  ...,  0.4574,  0.4023,  0.7730],
       grad_fn=<SliceBackward0>)